# Сбор данных со страницы VK с помощью API

In [12]:
# Задаем версию API ВКонтакте
version = '5.199'

# Читаем токен доступа из файла
with open('token.txt') as f:
    token = f.read()

# Читаем идентификатор пользователя из файла
with open('id_user.txt') as f:
    id_user = f.read()

In [13]:
# Импортируем необходимые библиотеки
import datetime
import sqlite3
import requests
import pandas as pd

In [15]:
# Определяем метод API для получения информации о пользователе
method = 'users.get'
parameters = f'user_ids={id_user}'

# Формируем URL для запроса к API
url = f'https://api.vk.com/method/{method}?{parameters}&v={version}&access_token={token}'

In [16]:
# Выполняем запрос и получаем ответ
response = requests.get(url)

In [18]:
# Определяем метод API для получения постов пользователя
method = 'wall.get'
parameters = f'owner_id={id_user}'  # ID пользователя как владельца стены
count = 100  # Количество постов для запроса

# Формируем URL для запроса к API
url = f'https://api.vk.com/method/{method}?{parameters}&v={version}&count={count}&access_token={token}'

# Выполняем запрос и получаем ответ
response = requests.get(url)

In [19]:
# Инициализируем список для хранения данных постов
data = []
for item in response.json()['response']['items']:
    post_id = item['id']  # Получаем ID поста
    post_time = datetime.datetime.fromtimestamp(item['date'])  # Конвертируем время поста
    likes = item['likes']['count']  # Получаем количество лайков

    # Определяем время суток поста
    if 6 <= post_time.hour < 12:
        time_of_day = 'morning'  # Утро
    elif 12 <= post_time.hour < 18:
        time_of_day = 'afternoon'  # День
    elif 18 <= post_time.hour < 24:
        time_of_day = 'evening'  # Вечер
    else:
        time_of_day = 'night'  # Ночь

    # Определяем день недели
    day_of_week = post_time.strftime('%a')  # Получаем сокращенное название дня
    # Переводим сокращенное название дня на английский
    day_of_week = {
        'Mon': 'Monday',
        'Tue': 'Tuesday',
        'Wed': 'Wednesday',
        'Thu': 'Thursday',
        'Fri': 'Friday',
        'Sat': 'Saturday',
        'Sun': 'Sunday'
    }[day_of_week]

    # Добавляем данные поста в список
    data.append({
        'id_post': post_id,
        'date_time': post_time,
        'likes': likes,
        'times_of_day': time_of_day,
        'day_of_the_week': day_of_week
    })

# Создаем DataFrame из собранных данных
df = pd.DataFrame(data)

# Печатаем содержимое DataFrame
print(df)

    id_post           date_time  likes times_of_day day_of_the_week
0      3024 2023-12-08 11:12:46     72      morning          Friday
1      3557 2024-10-07 14:34:07     55    afternoon          Monday
2      3554 2024-10-06 13:56:20      2    afternoon          Sunday
3      3553 2024-10-05 15:14:33     20    afternoon        Saturday
4      3548 2024-10-04 04:15:10     32        night          Friday
..      ...                 ...    ...          ...             ...
95     2945 2023-10-22 07:02:19    204      morning          Sunday
96     2944 2023-10-06 10:18:16      9      morning          Friday
97     2939 2023-09-22 12:40:47     55    afternoon          Friday
98     2935 2023-09-21 08:22:08     49      morning        Thursday
99     2921 2023-09-12 13:47:07    114    afternoon         Tuesday

[100 rows x 5 columns]


# Сохранение данных в csv файл

In [9]:
# Сохраняем в CSV файл
df.to_csv('posts_data.csv', index=False)

# Создание базы данных SQLite

In [10]:
# Создание подключения к SQLite базе данных
conn = sqlite3.connect('posts.db')

# Запись данных в SQLite
df.to_sql('posts', conn, if_exists='replace', index=False)

# Закрытие соединения
conn.close()

# SQL запрос для анализа влияния

In [11]:
# Подключение к базе данных
conn = sqlite3.connect('posts.db')
cursor = conn.cursor()

# SQL-запрос
query = """
WITH PostDifferences AS (
    -- Выбираем необходимые поля и вычисляем разницу в днях между текущей и предыдущей датой поста
    SELECT
        id_post,
        date_time,
        likes,
        times_of_day,
        day_of_the_week,
        JULIANDAY(date_time) - LAG(JULIANDAY(date_time)) OVER (ORDER BY date_time) AS gap_days
    FROM
        posts
)

-- Основной запрос для агрегирования данных по времени суток и дню недели
SELECT
    times_of_day,            -- Время суток
    day_of_the_week,        -- День недели
    AVG(likes) AS avg_likes, -- Среднее количество лайков
    AVG(gap_days) AS avg_gap_days -- Средний промежуток между постами в днях
FROM
    PostDifferences
GROUP BY
    times_of_day,           -- Группируем по времени суток
    day_of_the_week         -- Группируем по дню недели
ORDER BY
    avg_likes DESC;         -- Сортируем по среднему количеству лайков в порядке убывания
"""

# Выполнение запроса и получение результата
cursor.execute(query)
rows = cursor.fetchall()

# Закрытие соединения
cursor.close()
conn.close()

# Вывод результата
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.DataFrame(rows, columns=('Идентификатор поста', 'Время и дата поста', 'Количество лайков', 'Промежуток в днях')))

   Идентификатор поста Время и дата поста  Количество лайков  \
0            afternoon             Sunday         107.500000   
1              morning             Sunday          61.750000   
2            afternoon            Tuesday          57.750000   
3            afternoon           Thursday          57.666667   
4            afternoon           Saturday          55.833333   
5              morning           Saturday          53.000000   
6            afternoon             Monday          49.500000   
7              morning           Thursday          49.000000   
8                night             Friday          49.000000   
9                night          Wednesday          47.800000   
10             morning            Tuesday          46.166667   
11             morning             Friday          46.000000   
12             morning          Wednesday          44.857143   
13           afternoon             Friday          38.125000   
14               night           Saturda

**Вывод**: Время суток с максимальным количеством лайков – это утро и вечер. Воскресенье и суббота приносят больше лайков. Есть заметные различия в среднем промежутке между постами, однако его влияние на количество лайков не столь очевидно.Главным фактором для повышения количества лайков можно считать время суток, особенно утренние и вечерние часы. День недели также играет важную роль, особенно в выходные. Промежутки между постами менее значительно влияют на лайки, хотя их оптимизация также может иметь значение.